# Übung - Metadaten suchen & filtern

In dieser Übung wollen wir mit Hilfe von SPARQL Metadaten suchen und filtern, wie im vorherigen Abschnitt gezeigt. Dabei unterstützen wir Dr. Amir Weber in seinem Untersuchungsauftrag, zu prüfen, welche offen verfügbaren Daten existieren, die dazu beitragen können, den Baumbestand in einer bestimmten Region zu ermitteln.

## Suche nach "Baumkataster"

Nachdem wir im letzten Abschnitt nach offenen Daten von deutschen Bereitstellern im Allgemeinen gesucht haben, gilt es nun, die Suche zu verfeinern bzw. die Suchergebnisse zu filtern. Dazu werden wir nun nach Datensätzen suchen, die das Wort "Baumkataster" im Titel beinhalten. Außerdem werden wir das Ergebnis auf einen Zeitabschnitt beschränken. Für die folgende Suche entscheidet sich Dr. Weber spontan für den Zeitraum 2010-2015. 

:::{admonition} Einführung in den Code
:class: hinweis, dropdown
Als erstes setzen wir mit `(%endpoint https://data.europa.eu/sparql)` wieder unsere Verknüpfung zur Schnittstelle des europäischen Datenportals data.europa.eu. 

Auch die PREFIXES kennen Sie bereits aus dem vorigen Abschnitt:

Mit dem PREFIX `(dct: <http://purl.org/dc/terms/>)` kürzen wir Verweise auf die angegebene Adresse (der Namensraum der Dublic Core Metadata Initiative) auf *dct* (dublin core term).

Mit dem PREFIX `(dcatde: <http://dcat-ap.de/def/dcatde/>)` kürzen wir Verweise aus dem Namensraum von DCAT-AP auf *dcatde* (data catalog vocabulary deutschland).

Mit den SELECT-Befehlen geben wir die Eigenschaften an, die wir sehen wollen. Sie werden in der Ausgabe als Spalten angezeigt.

In dieser Übung kommen die `FILTER`-Befehle neu hinzu. Diese sind zusätzliche Bedingungen, die die Datensätze erfüllen müssen, um aufgelistet zu werden bzw. um Ergebnisse ein- oder auszuschließen. Sie werden als `FILTER(besipiel)` formuliert und in der Regel verwendet, um spezifische Merkmale von Daten zu wählen wie beispielsweise numerische Bedingungen (z. B. Werte größer oder kleiner als eine bestimmte Zahl).

So gibt der Befehl `FILTER(isURI(?contributorid))` beispielsweise an, dass nur Ergebnisse angezeigt werden sollen, bei denen der Bereitsteller als URI-Link angegeben ist. Damit können wir sicherstellen, dass eine Ressource eine identifizierbare Webadresse hat. Dies ist besonders in Bezug auf Datenvalidierung und -bereinigung von Bedeutung.

Der Befehl `FILTER(strstarts(str(?contributorid), "http://dcat-ap.de/def/contributors/"))` ist komplizierter, denn er bedeutet, dass nur jene Ergebnisse `?contributorid` berücksichtigt werden, deren URI-Zeichenkette mit `"http://dcat-ap.de/def/contributors/"` beginnt. D. h. er filtert die Datensätze so, dass nur diejenigen einbezogen werden, die als Contributor-IDs einen URI haben, der zu der spezifischen Sammlung oder Definition von Contributor-URIs gehören, die unter `http://dcat-ap.de/def/contributors/` zu finden sind. Das ist wichtig, denn wir werden sehen, dass die Namen der Datenbereitsteller im URI-Format angegeben sind. Die Funktion `strstarts` zusammen mit dem angegebenen Wert legt fest, mit welcher Zeichenfolge die Angabe beginnen muss. `str()` konvertiert ein URI oder einen anderen RDF-Term in eine Zeichenkette, sodass Zeichenkettenoperationen ausgeführt werden können. In unserem Fall bedeutet das, dass `str(?contributorid)` verwendet wird, um sicherzustellen, dass `?contributorid` als Zeichenkette behandelt wird.

Den Befehl `FILTER(CONTAINS(LCASE(?title), "baumkataster"))` verwenden wir, um die Ergebnisse so zu filtern, sodass nur die Einträge einbezogen werden, deren Titel die Zeichenfolge "baumkataster" enthalten, unabhängig von der Groß- oder Kleinschreibung. `CONTAINS()` ist der Befehl, der prüft, ob eine Zeichenkette einen bestimmten Teilstring enthält. `LCASE()` konvertiert alle Zeichen in einer Zeichenkette in Kleinbuchstaben. So wird die Variable `?title` in Kleinbuchstaben umgewandelt, bevor sie auf das Vorhandensein des Teilstrings überprüft wird. So stellen wir sicher, dass keine Einträge ausgeschlossen werden, nur weil "Baumkataster" in einer anderen Groß- und Kleinschreibung geschrieben ist, wie beispielsweise "BaumKataster" oder "BAUMKATASTER".

Um unsere Suche auf einen Zeitraum einzugrenzen, können wir ebenfalls den `Filter`-Befehl verwenden.

`STR()` konvertiert den Wert der Variable `?modified` in eine Zeichenkette. Dies ist notwendig, um die Zeichenfolgenoperationen ausführen zu können. Mit `CONTAINS()` stellen wir sicher, dasss eine Zeichenkette einen bestimmten Teilstring enthält. In unserem Fall prüft sie, ob die Zeichenkette, die das modifizierte Datum darstellt, einen der Teilstrings "2010", "2011", "2012", "2013", "2014" oder "2015" enthält. Die Verwendung von `||` verknüpft mehrere `CONTAINS`-Bedingungen mit einem logischen OR-Operator. Dies bedeutet, dass das Ergebnis akzeptiert wird, wenn eine der Bedingungen wahr ist.

Unsere Eingabe filtert die Ergebnisse also so, dass nur Einträge, deren `?modified`-Datum eine der genannten Jahreszahlen enthält, in den Ergebnissen verbleiben. Wenn also das Datum (z.B. in einem Zeitstempel wie `2013-05-15T13:45:30`) einen der Jahre zwischen 2010 und 2015 enthält, wird das Ergebnis beibehalten.
:::

**Code**

In [ ]:
%endpoint https://data.europa.eu/sparql
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>

SELECT ?uri ?title ?contributorid ?modified WHERE {
    {
        SELECT ?uri ?title ?contributorid ?modified WHERE {
            ?uri dct:title ?title .
            ?uri dcatde:contributorID ?contributorid .
            ?uri dct:modified ?modified .
            FILTER(isURI(?contributorid))
            FILTER(strstarts(str(?contributorid), "http://dcat-ap.de/def/contributors/"))       
            FILTER(CONTAINS(LCASE(?title), "baumkataster"))
            FILTER(
                CONTAINS(STR(?modified), "2010") || 
                CONTAINS(STR(?modified), "2011") || 
                CONTAINS(STR(?modified), "2012") || 
                CONTAINS(STR(?modified), "2013") || 
                CONTAINS(STR(?modified), "2014") || 
                CONTAINS(STR(?modified), "2015") 
                )
        }
    }
}

**Output**

In [ ]:
%endpoint https://data.europa.eu/sparql
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>

SELECT ?uri ?title ?contributorid ?modified WHERE {
    {
        SELECT ?uri ?title ?contributorid ?modified WHERE {
            ?uri dct:title ?title .
            ?uri dcatde:contributorID ?contributorid .
            ?uri dct:modified ?modified .
            FILTER(isURI(?contributorid))
            FILTER(strstarts(str(?contributorid), "http://dcat-ap.de/def/contributors/"))       
            FILTER(CONTAINS(LCASE(?title), "baumkataster"))
            FILTER(
                CONTAINS(STR(?modified), "2010") || 
                CONTAINS(STR(?modified), "2011") || 
                CONTAINS(STR(?modified), "2012") || 
                CONTAINS(STR(?modified), "2013") || 
                CONTAINS(STR(?modified), "2014") || 
                CONTAINS(STR(?modified), "2015") 
                )
        }
    }
}

**Erklärung des Ergebnissess**

Mit dieser Abfrage haben wir nach sämtlichen Datensätzen gesucht, die von einem deutschen Bereitsteller stammen, das Wort "Baumkataster" beinhalten und im Zeitraum 2010-2015 veröffentlicht wurden. Leider müssen wir feststellen, dass uns gar keine Daten ausgegeben werden.

## Anpassen der Abfrage

Nachdem er sich die Ausgabe angesehen hat, fällt Dr. Amir Weber ein, dass im Sommer 2017 das sog. Open-Data-Gesetz in Kraft getreten ist. Damit wurden Behörden der unmittelbaren Bundesverwaltung dazu verpflichtet, Datensätze "open", also offen und für alle zugänglich, bereitzustellen.

Er vermutet daher einen Anstieg der eingepflegten Daten nach 2017 und passt seine SPARQL-Abrage dementsprechend an. Aus diesem Grund wollen wir nun Daten aus den Jahren 2015 bis 2025 suchen.

:::{admonition} Einführung in den Code
:class: hinweis, dropdown
An der Grundstruktur des Codes brauchen wir nichts zu ändern. Es müssen lediglich die Jahreszahlen angepasst werden. Statt des Zeitraums 2010-2015 wird nun der Zeitraum 2015-2025 untersucht.
:::

**Code**

In [ ]:
%endpoint https://data.europa.eu/sparql
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>

SELECT ?uri ?title ?contributorid ?modified WHERE {
    {
        SELECT ?uri ?title ?contributorid ?modified WHERE {
            ?uri dct:title ?title .
            ?uri dcatde:contributorID ?contributorid .
            ?uri dct:modified ?modified .
            FILTER(isURI(?contributorid))
            FILTER(strstarts(str(?contributorid), "http://dcat-ap.de/def/contributors/"))       
            FILTER(CONTAINS(LCASE(?title), "baumkataster"))
            FILTER(
                CONTAINS(STR(?modified), "2015") ||
                CONTAINS(STR(?modified), "2016") ||
                CONTAINS(STR(?modified), "2017") ||
                CONTAINS(STR(?modified), "2018") ||
                CONTAINS(STR(?modified), "2019") ||
                CONTAINS(STR(?modified), "2020") ||
                CONTAINS(STR(?modified), "2021") ||
                CONTAINS(STR(?modified), "2022") ||
                CONTAINS(STR(?modified), "2023") ||
                CONTAINS(STR(?modified), "2024") ||
                CONTAINS(STR(?modified), "2025")
            )
        }
    }
}


**Output**

In [ ]:
%endpoint https://data.europa.eu/sparql
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>

SELECT ?uri ?title ?contributorid ?modified WHERE {
    {
        SELECT ?uri ?title ?contributorid ?modified WHERE {
            ?uri dct:title ?title .
            ?uri dcatde:contributorID ?contributorid .
            ?uri dct:modified ?modified .
            FILTER(isURI(?contributorid))
            FILTER(strstarts(str(?contributorid), "http://dcat-ap.de/def/contributors/"))       
            FILTER(CONTAINS(LCASE(?title), "baumkataster"))
            FILTER(
                CONTAINS(STR(?modified), "2015") ||
                CONTAINS(STR(?modified), "2016") ||
                CONTAINS(STR(?modified), "2017") ||
                CONTAINS(STR(?modified), "2018") ||
                CONTAINS(STR(?modified), "2019") ||
                CONTAINS(STR(?modified), "2020") ||
                CONTAINS(STR(?modified), "2021") ||
                CONTAINS(STR(?modified), "2022") ||
                CONTAINS(STR(?modified), "2023") ||
                CONTAINS(STR(?modified), "2024") ||
                CONTAINS(STR(?modified), "2025")
            )
        }
    }
}


Endpoint set to: https://data.europa.eu/sparql

uri,title,contributorid,modified
http://data.europa.eu/88u/dataset/fcdceb2e-d16d-410c-ba0f-521ba8c6effa,Fachpläne - Baumkataster,http://dcat-ap.de/def/contributors/datenBW,2025-01-03T08:49:41.490922
http://data.europa.eu/88u/dataset/c1c61928-c602-4e37-af31-2d23901e2540,Straßenbaumkataster Hamburg,http://dcat-ap.de/def/contributors/transparenzportalHamburg,2025-01-25T23:05:06.709577
http://data.europa.eu/88u/dataset/19676799-fedb-4d1a-a89a-26fba887b3f0~~2,Straßenbaumkataster Hamburger Hafen,http://dcat-ap.de/def/contributors/transparenzportalHamburg,2025-01-24T21:38:49.974593
http://data.europa.eu/88u/dataset/fb87ef77-86bf-444f-8f43-d73ef0708e42,D30 Baumkataster der Kreisstadt Bergheim,http://dcat-ap.de/def/contributors/openNRW,2021-09-08T08:02:44+02:00
http://data.europa.eu/88u/dataset/5f0aaf45-f380-49b3-87be-51abaddacbe0,Baumkataster Norderstedt,http://dcat-ap.de/def/contributors/schleswigHolstein,2022-04-12T07:58:22.519048
http://data.europa.eu/88u/dataset/cfbe70e6-6078-42db-8c9b-f8745ee8916e,Baumkataster,http://dcat-ap.de/def/contributors/schleswigHolstein,2023-06-09T05:25:52.631385
http://data.europa.eu/88u/dataset/b16bb333-26ca-4743-9663-723d63f57259,Digitales Baumkataster Münster,http://dcat-ap.de/def/contributors/openNRW,2025-03-13T13:28:04+01:00
http://data.europa.eu/88u/dataset/200e532c-a381-4611-ba05-4ed7afc31230,Baumkataster des ASG Wesel,http://dcat-ap.de/def/contributors/openNRW,2024-05-28T11:46:06.401383
http://data.europa.eu/88u/dataset/baumkataster_stadt_wuerzburg-wuerzburg,Baumkataster der Stadt Würzburg,http://dcat-ap.de/def/contributors/openDataBayern,2024-11-14T13:33:38.241Z
http://data.europa.eu/88u/dataset/https-registry-gdi-de-org-id-de-sn-stlp-cee75d75-4b4e-4bea-9d24-8b5993b54479,"Baumkataster, Stadt Leipzig",http://dcat-ap.de/def/contributors/freistaatSachsen,2025-02-19


**Erklärung des Ergebnisses**


Wir bekommen nun 43 Ergebnisse angegeben, von denen uns 20 angezeigt werden.

Wie Dr. Weber richtig vermutet hat, sind die meisten Datensätze noch nicht allzu lange online verfügbar.


## Zusammenfassung

In diesem Abschnitt haben wir uns dem Forschungsauftrag von Dr. Weber gewidmet und unsere Suche nach offenen deutschen Behördendaten auf solche eingegrenzt, die das Wort Baumkataster im Titel tragen. Anschließend haben wir die Suche auf einen bestimmten Zeitraum eingegrenzt und diesen verschoben. 

Dabei haben wir die Kenntnisse der Befehle PREFIX, SELECT und WHERE gefestigt und die FILTER-Funktion kennengelernt.

Im nächsten Abschnitt wird die Anzeige des Ergebnisses so angepasst, dass wir uns anzeigen lassen können, welche Datenbereitsteller am produktivsten sind und welche Formate verwendet werden.